In [1]:
from ortools.linear_solver import pywraplp
from ortools.init import pywrapinit

In [2]:
def main():
    # Create the linear solver with the GLOP backend.
    solver = pywraplp.Solver.CreateSolver('GLOP')
    if not solver:
        return

    # Create the variables x and y.
    x = solver.NumVar(0, 1, 'x')
    y = solver.NumVar(0, 2, 'y')

    print('Number of variables =', solver.NumVariables())

    # Create a linear constraint, 0 <= x + y <= 2.
    ct = solver.Constraint(0, 2, 'ct')
    ct.SetCoefficient(x, 1)
    ct.SetCoefficient(y, 1)

    print('Number of constraints =', solver.NumConstraints())

    # Create the objective function, 3 * x + y.
    objective = solver.Objective()
    objective.SetCoefficient(x, 3)
    objective.SetCoefficient(y, 1)
    objective.SetMaximization()

    solver.Solve()

    print('Solution:')
    print('Objective value =', objective.Value())
    print('x =', x.solution_value())
    print('y =', y.solution_value())

In [3]:
pywrapinit.CppBridge.InitLogging('basic_example.py')
cpp_flags = pywrapinit.CppFlags()
cpp_flags.logtostderr = True
cpp_flags.log_prefix = False
pywrapinit.CppBridge.SetFlags(cpp_flags)

In [4]:
main()

Number of variables = 2
Number of constraints = 1
Solution:
Objective value = 4.0
x = 1.0
y = 1.0


In [5]:
from ortools.linear_solver import pywraplp

In [6]:
# Nutrient minimums.
nutrients = [
    ['Calories (kcal)', 3],
    ['Protein (g)', 70],
    ['Calcium (g)', 0.8],
    ['Iron (mg)', 12],
    ['Vitamin A (KIU)', 5],
    ['Vitamin B1 (mg)', 1.8],
    ['Vitamin B2 (mg)', 2.7],
    ['Niacin (mg)', 18],
    ['Vitamin C (mg)', 75],
]

# Commodity, Unit, 1939 price (cents), Calories (kcal), Protein (g),
# Calcium (g), Iron (mg), Vitamin A (KIU), Vitamin B1 (mg), Vitamin B2 (mg),
# Niacin (mg), Vitamin C (mg)
data = [
    [
        'Wheat Flour (Enriched)', '10 lb.', 36, 44.7, 1411, 2, 365, 0, 55.4,
        33.3, 441, 0
    ],
    ['Macaroni', '1 lb.', 14.1, 11.6, 418, 0.7, 54, 0, 3.2, 1.9, 68, 0],
    [
        'Wheat Cereal (Enriched)', '28 oz.', 24.2, 11.8, 377, 14.4, 175, 0,
        14.4, 8.8, 114, 0
    ],
    ['Corn Flakes', '8 oz.', 7.1, 11.4, 252, 0.1, 56, 0, 13.5, 2.3, 68, 0],
    [
        'Corn Meal', '1 lb.', 4.6, 36.0, 897, 1.7, 99, 30.9, 17.4, 7.9, 106,
        0
    ],
    [
        'Hominy Grits', '24 oz.', 8.5, 28.6, 680, 0.8, 80, 0, 10.6, 1.6,
        110, 0
    ],
    ['Rice', '1 lb.', 7.5, 21.2, 460, 0.6, 41, 0, 2, 4.8, 60, 0],
    ['Rolled Oats', '1 lb.', 7.1, 25.3, 907, 5.1, 341, 0, 37.1, 8.9, 64, 0],
    [
        'White Bread (Enriched)', '1 lb.', 7.9, 15.0, 488, 2.5, 115, 0,
        13.8, 8.5, 126, 0
    ],
    [
        'Whole Wheat Bread', '1 lb.', 9.1, 12.2, 484, 2.7, 125, 0, 13.9,
        6.4, 160, 0
    ],
    ['Rye Bread', '1 lb.', 9.1, 12.4, 439, 1.1, 82, 0, 9.9, 3, 66, 0],
    ['Pound Cake', '1 lb.', 24.8, 8.0, 130, 0.4, 31, 18.9, 2.8, 3, 17, 0],
    ['Soda Crackers', '1 lb.', 15.1, 12.5, 288, 0.5, 50, 0, 0, 0, 0, 0],
    ['Milk', '1 qt.', 11, 6.1, 310, 10.5, 18, 16.8, 4, 16, 7, 177],
    [
        'Evaporated Milk (can)', '14.5 oz.', 6.7, 8.4, 422, 15.1, 9, 26, 3,
        23.5, 11, 60
    ],
    ['Butter', '1 lb.', 30.8, 10.8, 9, 0.2, 3, 44.2, 0, 0.2, 2, 0],
    ['Oleomargarine', '1 lb.', 16.1, 20.6, 17, 0.6, 6, 55.8, 0.2, 0, 0, 0],
    ['Eggs', '1 doz.', 32.6, 2.9, 238, 1.0, 52, 18.6, 2.8, 6.5, 1, 0],
    [
        'Cheese (Cheddar)', '1 lb.', 24.2, 7.4, 448, 16.4, 19, 28.1, 0.8,
        10.3, 4, 0
    ],
    ['Cream', '1/2 pt.', 14.1, 3.5, 49, 1.7, 3, 16.9, 0.6, 2.5, 0, 17],
    [
        'Peanut Butter', '1 lb.', 17.9, 15.7, 661, 1.0, 48, 0, 9.6, 8.1,
        471, 0
    ],
    ['Mayonnaise', '1/2 pt.', 16.7, 8.6, 18, 0.2, 8, 2.7, 0.4, 0.5, 0, 0],
    ['Crisco', '1 lb.', 20.3, 20.1, 0, 0, 0, 0, 0, 0, 0, 0],
    ['Lard', '1 lb.', 9.8, 41.7, 0, 0, 0, 0.2, 0, 0.5, 5, 0],
    [
        'Sirloin Steak', '1 lb.', 39.6, 2.9, 166, 0.1, 34, 0.2, 2.1, 2.9,
        69, 0
    ],
    ['Round Steak', '1 lb.', 36.4, 2.2, 214, 0.1, 32, 0.4, 2.5, 2.4, 87, 0],
    ['Rib Roast', '1 lb.', 29.2, 3.4, 213, 0.1, 33, 0, 0, 2, 0, 0],
    ['Chuck Roast', '1 lb.', 22.6, 3.6, 309, 0.2, 46, 0.4, 1, 4, 120, 0],
    ['Plate', '1 lb.', 14.6, 8.5, 404, 0.2, 62, 0, 0.9, 0, 0, 0],
    [
        'Liver (Beef)', '1 lb.', 26.8, 2.2, 333, 0.2, 139, 169.2, 6.4, 50.8,
        316, 525
    ],
    ['Leg of Lamb', '1 lb.', 27.6, 3.1, 245, 0.1, 20, 0, 2.8, 3.9, 86, 0],
    [
        'Lamb Chops (Rib)', '1 lb.', 36.6, 3.3, 140, 0.1, 15, 0, 1.7, 2.7,
        54, 0
    ],
    ['Pork Chops', '1 lb.', 30.7, 3.5, 196, 0.2, 30, 0, 17.4, 2.7, 60, 0],
    [
        'Pork Loin Roast', '1 lb.', 24.2, 4.4, 249, 0.3, 37, 0, 18.2, 3.6,
        79, 0
    ],
    ['Bacon', '1 lb.', 25.6, 10.4, 152, 0.2, 23, 0, 1.8, 1.8, 71, 0],
    ['Ham, smoked', '1 lb.', 27.4, 6.7, 212, 0.2, 31, 0, 9.9, 3.3, 50, 0],
    ['Salt Pork', '1 lb.', 16, 18.8, 164, 0.1, 26, 0, 1.4, 1.8, 0, 0],
    [
        'Roasting Chicken', '1 lb.', 30.3, 1.8, 184, 0.1, 30, 0.1, 0.9, 1.8,
        68, 46
    ],
    ['Veal Cutlets', '1 lb.', 42.3, 1.7, 156, 0.1, 24, 0, 1.4, 2.4, 57, 0],
    [
        'Salmon, Pink (can)', '16 oz.', 13, 5.8, 705, 6.8, 45, 3.5, 1, 4.9,
        209, 0
    ],
    ['Apples', '1 lb.', 4.4, 5.8, 27, 0.5, 36, 7.3, 3.6, 2.7, 5, 544],
    ['Bananas', '1 lb.', 6.1, 4.9, 60, 0.4, 30, 17.4, 2.5, 3.5, 28, 498],
    ['Lemons', '1 doz.', 26, 1.0, 21, 0.5, 14, 0, 0.5, 0, 4, 952],
    ['Oranges', '1 doz.', 30.9, 2.2, 40, 1.1, 18, 11.1, 3.6, 1.3, 10, 1998],
    ['Green Beans', '1 lb.', 7.1, 2.4, 138, 3.7, 80, 69, 4.3, 5.8, 37, 862],
    ['Cabbage', '1 lb.', 3.7, 2.6, 125, 4.0, 36, 7.2, 9, 4.5, 26, 5369],
    ['Carrots', '1 bunch', 4.7, 2.7, 73, 2.8, 43, 188.5, 6.1, 4.3, 89, 608],
    ['Celery', '1 stalk', 7.3, 0.9, 51, 3.0, 23, 0.9, 1.4, 1.4, 9, 313],
    ['Lettuce', '1 head', 8.2, 0.4, 27, 1.1, 22, 112.4, 1.8, 3.4, 11, 449],
    ['Onions', '1 lb.', 3.6, 5.8, 166, 3.8, 59, 16.6, 4.7, 5.9, 21, 1184],
    [
        'Potatoes', '15 lb.', 34, 14.3, 336, 1.8, 118, 6.7, 29.4, 7.1, 198,
        2522
    ],
    ['Spinach', '1 lb.', 8.1, 1.1, 106, 0, 138, 918.4, 5.7, 13.8, 33, 2755],
    [
        'Sweet Potatoes', '1 lb.', 5.1, 9.6, 138, 2.7, 54, 290.7, 8.4, 5.4,
        83, 1912
    ],
    [
        'Peaches (can)', 'No. 2 1/2', 16.8, 3.7, 20, 0.4, 10, 21.5, 0.5, 1,
        31, 196
    ],
    [
        'Pears (can)', 'No. 2 1/2', 20.4, 3.0, 8, 0.3, 8, 0.8, 0.8, 0.8, 5,
        81
    ],
    [
        'Pineapple (can)', 'No. 2 1/2', 21.3, 2.4, 16, 0.4, 8, 2, 2.8, 0.8,
        7, 399
    ],
    [
        'Asparagus (can)', 'No. 2', 27.7, 0.4, 33, 0.3, 12, 16.3, 1.4, 2.1,
        17, 272
    ],
    [
        'Green Beans (can)', 'No. 2', 10, 1.0, 54, 2, 65, 53.9, 1.6, 4.3,
        32, 431
    ],
    [
        'Pork and Beans (can)', '16 oz.', 7.1, 7.5, 364, 4, 134, 3.5, 8.3,
        7.7, 56, 0
    ],
    ['Corn (can)', 'No. 2', 10.4, 5.2, 136, 0.2, 16, 12, 1.6, 2.7, 42, 218],
    [
        'Peas (can)', 'No. 2', 13.8, 2.3, 136, 0.6, 45, 34.9, 4.9, 2.5, 37,
        370
    ],
    [
        'Tomatoes (can)', 'No. 2', 8.6, 1.3, 63, 0.7, 38, 53.2, 3.4, 2.5,
        36, 1253
    ],
    [
        'Tomato Soup (can)', '10 1/2 oz.', 7.6, 1.6, 71, 0.6, 43, 57.9, 3.5,
        2.4, 67, 862
    ],
    [
        'Peaches, Dried', '1 lb.', 15.7, 8.5, 87, 1.7, 173, 86.8, 1.2, 4.3,
        55, 57
    ],
    [
        'Prunes, Dried', '1 lb.', 9, 12.8, 99, 2.5, 154, 85.7, 3.9, 4.3, 65,
        257
    ],
    [
        'Raisins, Dried', '15 oz.', 9.4, 13.5, 104, 2.5, 136, 4.5, 6.3, 1.4,
        24, 136
    ],
    [
        'Peas, Dried', '1 lb.', 7.9, 20.0, 1367, 4.2, 345, 2.9, 28.7, 18.4,
        162, 0
    ],
    [
        'Lima Beans, Dried', '1 lb.', 8.9, 17.4, 1055, 3.7, 459, 5.1, 26.9,
        38.2, 93, 0
    ],
    [
        'Navy Beans, Dried', '1 lb.', 5.9, 26.9, 1691, 11.4, 792, 0, 38.4,
        24.6, 217, 0
    ],
    ['Coffee', '1 lb.', 22.4, 0, 0, 0, 0, 0, 4, 5.1, 50, 0],
    ['Tea', '1/4 lb.', 17.4, 0, 0, 0, 0, 0, 0, 2.3, 42, 0],
    ['Cocoa', '8 oz.', 8.6, 8.7, 237, 3, 72, 0, 2, 11.9, 40, 0],
    ['Chocolate', '8 oz.', 16.2, 8.0, 77, 1.3, 39, 0, 0.9, 3.4, 14, 0],
    ['Sugar', '10 lb.', 51.7, 34.9, 0, 0, 0, 0, 0, 0, 0, 0],
    ['Corn Syrup', '24 oz.', 13.7, 14.7, 0, 0.5, 74, 0, 0, 0, 5, 0],
    ['Molasses', '18 oz.', 13.6, 9.0, 0, 10.3, 244, 0, 1.9, 7.5, 146, 0],
    [
        'Strawberry Preserves', '1 lb.', 20.5, 6.4, 11, 0.4, 7, 0.2, 0.2,
        0.4, 3, 0
    ],
]

In [7]:
# Instantiate a Glop solver and naming it.
solver = pywraplp.Solver.CreateSolver('GLOP')

In [8]:
solver

<ortools.linear_solver.pywraplp.Solver; proxy of <Swig Object of type 'operations_research::MPSolver *' at 0x000001D663589180> >

In [9]:
# Declare an array to hold our variables.
foods = [solver.NumVar(0.0, solver.infinity(), item[0]) for item in data]

print('Number of variables =', solver.NumVariables())

Number of variables = 77


In [10]:
# Create the constraints, one per nutrient.
constraints = []
for i, nutrient in enumerate(nutrients):
    constraints.append(solver.Constraint(nutrient[1], solver.infinity()))
    for j, item in enumerate(data):
        constraints[i].SetCoefficient(foods[j], item[i + 3])

print('Number of constraints =', solver.NumConstraints())

Number of constraints = 9


In [11]:
nutrients

[['Calories (kcal)', 3],
 ['Protein (g)', 70],
 ['Calcium (g)', 0.8],
 ['Iron (mg)', 12],
 ['Vitamin A (KIU)', 5],
 ['Vitamin B1 (mg)', 1.8],
 ['Vitamin B2 (mg)', 2.7],
 ['Niacin (mg)', 18],
 ['Vitamin C (mg)', 75]]

In [12]:
constraints

[<ortools.linear_solver.pywraplp.Constraint; proxy of <Swig Object of type 'operations_research::MPConstraint *' at 0x000001D663656F00> >,
 <ortools.linear_solver.pywraplp.Constraint; proxy of <Swig Object of type 'operations_research::MPConstraint *' at 0x000001D663656630> >,
 <ortools.linear_solver.pywraplp.Constraint; proxy of <Swig Object of type 'operations_research::MPConstraint *' at 0x000001D663647CC0> >,
 <ortools.linear_solver.pywraplp.Constraint; proxy of <Swig Object of type 'operations_research::MPConstraint *' at 0x000001D663647E70> >,
 <ortools.linear_solver.pywraplp.Constraint; proxy of <Swig Object of type 'operations_research::MPConstraint *' at 0x000001D663647F00> >,
 <ortools.linear_solver.pywraplp.Constraint; proxy of <Swig Object of type 'operations_research::MPConstraint *' at 0x000001D663589510> >,
 <ortools.linear_solver.pywraplp.Constraint; proxy of <Swig Object of type 'operations_research::MPConstraint *' at 0x000001D663589720> >,
 <ortools.linear_solver.pyw

In [13]:
# Objective function: Minimize the sum of (price-normalized) foods.
objective = solver.Objective()
for food in foods:
    objective.SetCoefficient(food, 1)
objective.SetMinimization()

In [14]:
status = solver.Solve()

In [15]:
solver.OPTIMAL

0

In [16]:
if status != solver.OPTIMAL:
    print('The problem does not have an optimal solution!')
    if status == solver.FEASIBLE:
        print('A potentially suboptimal solution was found.')
    else:
        print('The solver could not solve the problem.')
        exit(1)

In [17]:
foods[0].solution_value()

0.029519061676488285

In [18]:
objective.SetMinimization??

In [19]:
solver = pywraplp.Solver.CreateSolver('SCIP')


In [20]:
# Display the amounts (in dollars) to purchase of each food.
nutrients_result = [0] * len(nutrients)
print('\nAnnual Foods:')
for i, food in enumerate(foods):
    if food.solution_value() > 0.0:
        print('{}: ${}'.format(data[i][0], 365. * food.solution_value()))
        for j, _ in enumerate(nutrients):
            nutrients_result[j] += data[i][j + 3] * food.solution_value()
print('\nOptimal annual price: ${:.4f}'.format(365. * objective.Value()))

print('\nNutrients per day:')
for i, nutrient in enumerate(nutrients):
    print('{}: {:.2f} (min {})'.format(nutrient[0], nutrients_result[i],
                                       nutrient[1]))


Annual Foods:
Wheat Flour (Enriched): $10.774457511918223
Liver (Beef): $0.6907834111074193
Cabbage: $4.093268864842877
Spinach: $1.8277960703546996
Navy Beans, Dried: $22.275425687243036

Optimal annual price: $39.6617

Nutrients per day:
Calories (kcal): 3.00 (min 3)
Protein (g): 147.41 (min 70)
Calcium (g): 0.80 (min 0.8)
Iron (mg): 60.47 (min 12)
Vitamin A (KIU): 5.00 (min 5)
Vitamin B1 (mg): 4.12 (min 1.8)
Vitamin B2 (mg): 2.70 (min 2.7)
Niacin (mg): 27.32 (min 18)
Vitamin C (mg): 75.00 (min 75)


In [21]:
solver.infinity??

In [22]:
solver.NumVar

SyntaxError: unexpected EOF while parsing (1694260023.py, line 1)

In [23]:
objective.SetCoefficient??

In [24]:
solver.Constraint??

In [25]:
nutrients = [
        ['Calories (kcal)', 3],
        ['Protein (g)', 70],
        ['Calcium (g)', 0.8],
        ['Iron (mg)', 12],
        ['Vitamin A (KIU)', 5],
        ['Vitamin B1 (mg)', 1.8],
        ['Vitamin B2 (mg)', 2.7],
        ['Niacin (mg)', 18],
        ['Vitamin C (mg)', 75],
    ]

    # Commodity, Unit, 1939 price (cents), Calories (kcal), Protein (g),
    # Calcium (g), Iron (mg), Vitamin A (KIU), Vitamin B1 (mg), Vitamin B2 (mg),
    # Niacin (mg), Vitamin C (mg)
    data = [
        [
            'Wheat Flour (Enriched)', '10 lb.', 36, 44.7, 1411, 2, 365, 0, 55.4,
            33.3, 441, 0

IndentationError: unexpected indent (2284710089.py, line 16)

In [26]:
# Create the mip solver with the SCIP backend.
solver = pywraplp.Solver.CreateSolver('SCIP')


In [27]:
infinity = solver.infinity()
# x and y are integer non-negative variables.
x = solver.IntVar(0.0, infinity, 'x')
y = solver.IntVar(0.0, infinity, 'y')

print('Number of variables =', solver.NumVariables())

Number of variables = 2


In [28]:
# x + 7 * y <= 17.5.
solver.Add(x + 7 * y <= 17.5)

# x <= 3.5.
solver.Add(x <= 3.5)

print('Number of constraints =', solver.NumConstraints())

Number of constraints = 2


In [29]:
# Maximize x + 10 * y.
solver.Maximize(x + 10 * y)

In [30]:
status = solver.Solve()

In [31]:
status

0

In [32]:
 pywraplp.Solver.OPTIMAL

0

In [33]:
if status == pywraplp.Solver.OPTIMAL:
    print('Solution:')
    print('Objective value =', solver.Objective().Value())
    print('x =', x.solution_value())
    print('y =', y.solution_value())
else:
    print('The problem does not have an optimal solution.')

Solution:
Objective value = 23.0
x = 3.0
y = 2.0


In [34]:
def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data['constraint_coeffs'] = [
        [5, 7, 9, 2, 1],
        [18, 4, -9, 10, 12],
        [4, 7, 3, 8, 5],
        [5, 13, 16, 3, -7],
    ]
    data['bounds'] = [250, 285, 211, 315]
    data['obj_coeffs'] = [7, 8, 2, 9, 6]
    data['num_vars'] = 5
    data['num_constraints'] = 4
    return data


In [35]:
data = create_data_model()

In [36]:
data

{'constraint_coeffs': [[5, 7, 9, 2, 1],
  [18, 4, -9, 10, 12],
  [4, 7, 3, 8, 5],
  [5, 13, 16, 3, -7]],
 'bounds': [250, 285, 211, 315],
 'obj_coeffs': [7, 8, 2, 9, 6],
 'num_vars': 5,
 'num_constraints': 4}

In [37]:
# Create the mip solver with the SCIP backend.
solver = pywraplp.Solver.CreateSolver('SCIP')

In [ ]:
infinity = solver.infinity()
x = {}
for j in range(data['num_vars']):
    x[j] = solver.IntVar(0, infinity, 'x[%i]' % j)
print('Number of variables =', solver.NumVariables())

In [ ]:
 'x[%i]' % 4

In [ ]:
for j in range(data['num_vars']):
    print(j)

In [72]:
for i in range(data['num_constraints']):
    constraint = solver.RowConstraint(0, data['bounds'][i], '')
    for j in range(data['num_vars']):
        constraint.SetCoefficient(x[j], data['constraint_coeffs'][i][j])
print('Number of constraints =', solver.NumConstraints())
# In Python, you can also set the constraints as follows.
# for i in range(data['num_constraints']):
#  constraint_expr = \
# [data['constraint_coeffs'][i][j] * x[j] for j in range(data['num_vars'])]
#  solver.Add(sum(constraint_expr) <= data['bounds'][i])

Number of constraints = 4


In [73]:
objective = solver.Objective()
for j in range(data['num_vars']):
    objective.SetCoefficient(x[j], data['obj_coeffs'][j])
objective.SetMaximization()
# In Python, you can also set the objective as follows.
# obj_expr = [data['obj_coeffs'][j] * x[j] for j in range(data['num_vars'])]
# solver.Maximize(solver.Sum(obj_expr))

In [74]:
status = solver.Solve()

In [133]:
if status == pywraplp.Solver.OPTIMAL:
    print('Objective value =', solver.Objective().Value())
    for j in range(data['num_vars']):
        print(x[j].name(), ' = ', x[j].solution_value())
    print()
    print('Problem solved in %f milliseconds' % solver.wall_time())
    print('Problem solved in %d iterations' % solver.iterations())
    print('Problem solved in %d branch-and-bound nodes' % solver.nodes())
else:
    print('The problem does not have an optimal solution.')

Objective value = 259.99999999999966
x[0]  =  8.0
x[1]  =  21.0
x[2]  =  0.0
x[3]  =  2.0
x[4]  =  3.0

Problem solved in 3416092.000000 milliseconds
Problem solved in 71 iterations
Problem solved in 7 branch-and-bound nodes


In [134]:
from ortools.sat.python import cp_model

In [136]:
model = cp_model.CpModel()

In [137]:
model

In [139]:
num_vals = 3
x = model.NewIntVar(0, num_vals - 1, 'x')
y = model.NewIntVar(0, num_vals - 1, 'y')
z = model.NewIntVar(0, num_vals - 1, 'z')

In [141]:
y

y(0..2)

In [142]:
model.Add(x != y)

In [143]:
model.Add(x != y)

In [145]:
solver = cp_model.CpSolver()
status = solver.Solve(model)

In [155]:
if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    print('x = %i' % solver.Value(x))
    print('y = %i' % solver.Value(y))
    print('z = %i' % solver.Value(z))
else:
    print('No solution found.')

x = 1
y = 0
z = 0


In [156]:
status

4

In [157]:
"""Simple solve."""
from ortools.sat.python import cp_model


def SimpleSatProgram():
    """Minimal CP-SAT example to showcase calling the solver."""
    # Creates the model.
    model = cp_model.CpModel()

    # Creates the variables.
    num_vals = 3
    x = model.NewIntVar(0, num_vals - 1, 'x')
    y = model.NewIntVar(0, num_vals - 1, 'y')
    z = model.NewIntVar(0, num_vals - 1, 'z')

    # Creates the constraints.
    model.Add(x != y)

    # Creates a solver and solves the model.
    solver = cp_model.CpSolver()
    status = solver.Solve(model)

    if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
        print('x = %i' % solver.Value(x))
        print('y = %i' % solver.Value(y))
        print('z = %i' % solver.Value(z))
    else:
        print('No solution found.')


SimpleSatProgram()

x = 1
y = 0
z = 0


In [158]:
class VarArraySolutionPrinter(cp_model.CpSolverSolutionCallback):
    """Print intermediate solutions."""

    def __init__(self, variables):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self.__variables = variables
        self.__solution_count = 0

    def on_solution_callback(self):
        self.__solution_count += 1
        for v in self.__variables:
            print('%s=%i' % (v, self.Value(v)), end=' ')
        print()

    def solution_count(self):
        return self.__solution_count

In [159]:
solver = cp_model.CpSolver()
solution_printer = VarArraySolutionPrinter([x, y, z])
# Enumerate all solutions.
solver.parameters.enumerate_all_solutions = True
# Solve.
status = solver.Solve(model, solution_printer)

x=1 y=0 z=0 
x=1 y=0 z=0 
x=1 y=0 z=0 
x=1 y=0 z=0 
x=1 y=0 z=0 
x=1 y=0 z=0 
x=1 y=0 z=0 
x=1 y=0 z=0 
x=2 y=0 z=0 
x=2 y=0 z=0 
x=2 y=0 z=0 
x=2 y=0 z=0 
x=2 y=0 z=0 
x=2 y=0 z=0 
x=2 y=0 z=0 
x=2 y=0 z=0 
x=2 y=0 z=1 
x=2 y=0 z=1 
x=2 y=0 z=1 
x=2 y=0 z=1 
x=2 y=0 z=1 
x=2 y=0 z=1 
x=2 y=0 z=1 
x=2 y=0 z=1 
x=1 y=0 z=1 
x=1 y=0 z=1 
x=1 y=0 z=1 
x=1 y=0 z=1 
x=1 y=0 z=1 
x=1 y=0 z=1 
x=1 y=0 z=1 
x=1 y=0 z=1 
x=1 y=0 z=1 
x=1 y=0 z=1 
x=1 y=0 z=1 
x=1 y=0 z=1 
x=2 y=0 z=1 
x=2 y=0 z=1 
x=2 y=0 z=1 
x=2 y=0 z=1 
x=2 y=0 z=0 
x=2 y=0 z=0 
x=2 y=0 z=0 
x=2 y=0 z=0 
x=1 y=0 z=0 
x=1 y=0 z=0 
x=1 y=0 z=0 
x=1 y=0 z=0 
x=1 y=0 z=0 
x=1 y=0 z=0 
x=2 y=0 z=0 
x=2 y=0 z=0 
x=2 y=0 z=1 
x=2 y=0 z=1 
x=1 y=0 z=1 
x=1 y=0 z=1 
x=1 y=0 z=1 
x=1 y=0 z=1 
x=1 y=0 z=1 
x=1 y=0 z=1 
x=2 y=0 z=1 
x=2 y=0 z=1 
x=2 y=0 z=1 
x=2 y=0 z=1 
x=2 y=0 z=0 
x=2 y=0 z=0 
x=2 y=0 z=0 
x=2 y=0 z=0 
x=1 y=0 z=0 
x=1 y=0 z=0 
x=1 y=0 z=0 
x=1 y=0 z=0 
x=1 y=0 z=0 
x=1 y=0 z=0 
x=2 y=0 z=0 
x=2 y=0 z=0 
x=2 y=0 z=1 

In [160]:
from ortools.sat.python import cp_model


class VarArraySolutionPrinter(cp_model.CpSolverSolutionCallback):
    """Print intermediate solutions."""

    def __init__(self, variables):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self.__variables = variables
        self.__solution_count = 0

    def on_solution_callback(self):
        self.__solution_count += 1
        for v in self.__variables:
            print('%s=%i' % (v, self.Value(v)), end=' ')
        print()

    def solution_count(self):
        return self.__solution_count


def SearchForAllSolutionsSampleSat():
    """Showcases calling the solver to search for all solutions."""
    # Creates the model.
    model = cp_model.CpModel()

    # Creates the variables.
    num_vals = 3
    x = model.NewIntVar(0, num_vals - 1, 'x')
    y = model.NewIntVar(0, num_vals - 1, 'y')
    z = model.NewIntVar(0, num_vals - 1, 'z')

    # Create the constraints.
    model.Add(x != y)

    # Create a solver and solve.
    solver = cp_model.CpSolver()
    solution_printer = VarArraySolutionPrinter([x, y, z])
    # Enumerate all solutions.
    solver.parameters.enumerate_all_solutions = True
    # Solve.
    status = solver.Solve(model, solution_printer)

    print('Status = %s' % solver.StatusName(status))
    print('Number of solutions found: %i' % solution_printer.solution_count())


SearchForAllSolutionsSampleSat()

x=1 y=0 z=0 
x=2 y=0 z=0 
x=2 y=0 z=1 
x=1 y=0 z=1 
x=2 y=1 z=1 
x=2 y=1 z=0 
x=2 y=1 z=2 
x=2 y=0 z=2 
x=1 y=0 z=2 
x=0 y=1 z=2 
x=0 y=1 z=1 
x=0 y=2 z=1 
x=0 y=2 z=2 
x=1 y=2 z=2 
x=1 y=2 z=1 
x=1 y=2 z=0 
x=0 y=2 z=0 
x=0 y=1 z=0 
Status = OPTIMAL
Number of solutions found: 18


In [161]:
"""Simple solve."""
from ortools.sat.python import cp_model


def main():
    """Minimal CP-SAT example to showcase calling the solver."""
    # Creates the model.
    model = cp_model.CpModel()

    # Creates the variables.
    var_upper_bound = max(50, 45, 37)
    x = model.NewIntVar(0, var_upper_bound, 'x')
    y = model.NewIntVar(0, var_upper_bound, 'y')
    z = model.NewIntVar(0, var_upper_bound, 'z')

    # Creates the constraints.
    model.Add(2 * x + 7 * y + 3 * z <= 50)
    model.Add(3 * x - 5 * y + 7 * z <= 45)
    model.Add(5 * x + 2 * y - 6 * z <= 37)

    model.Maximize(2 * x + 2 * y + 3 * z)

    # Creates a solver and solves the model.
    solver = cp_model.CpSolver()
    status = solver.Solve(model)

    if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
        print(f'Maximum of objective function: {solver.ObjectiveValue()}\n')
        print(f'x = {solver.Value(x)}')
        print(f'y = {solver.Value(y)}')
        print(f'z = {solver.Value(z)}')
    else:
        print('No solution found.')

    # Statistics.
    print('\nStatistics')
    print(f'  status   : {solver.StatusName(status)}')
    print(f'  conflicts: {solver.NumConflicts()}')
    print(f'  branches : {solver.NumBranches()}')
    print(f'  wall time: {solver.WallTime()} s')


if __name__ == '__main__':
    main()

Maximum of objective function: 35.0

x = 7
y = 3
z = 5

Statistics
  status   : OPTIMAL
  conflicts: 2
  branches : 1
  wall time: 0.023142100000000002 s


In [162]:
from ortools.sat.python import cp_model

In [163]:
model = cp_model.CpModel()

In [164]:
base = 10

c = model.NewIntVar(1, base - 1, 'C')
p = model.NewIntVar(0, base - 1, 'P')
i = model.NewIntVar(1, base - 1, 'I')
s = model.NewIntVar(0, base - 1, 'S')
f = model.NewIntVar(1, base - 1, 'F')
u = model.NewIntVar(0, base - 1, 'U')
n = model.NewIntVar(0, base - 1, 'N')
t = model.NewIntVar(1, base - 1, 'T')
r = model.NewIntVar(0, base - 1, 'R')
e = model.NewIntVar(0, base - 1, 'E')

# We need to group variables in a list to use the constraint AllDifferent.
letters = [c, p, i, s, f, u, n, t, r, e]

# Verify that we have enough digits.
assert base >= len(letters)

In [167]:
model.AddAllDifferent(letters)

# CP + IS + FUN = TRUE
model.Add(c * base + p + i * base + s + f * base * base + u * base +
          n == t * base * base * base + r * base * base + u * base + e)

In [169]:
class VarArraySolutionPrinter(cp_model.CpSolverSolutionCallback):
    """Print intermediate solutions."""

    def __init__(self, variables):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self.__variables = variables
        self.__solution_count = 0

    def on_solution_callback(self):
        self.__solution_count += 1
        for v in self.__variables:
            print('%s=%i' % (v, self.Value(v)), end=' ')
        print()

    def solution_count(self):
        return self.__solution_count 

In [170]:
solver = cp_model.CpSolver()
solution_printer = VarArraySolutionPrinter(letters)
# Enumerate all solutions.
solver.parameters.enumerate_all_solutions = True
# Solve.
status = solver.Solve(model, solution_printer)

C=6 P=4 I=3 S=5 F=9 U=2 N=8 T=1 R=0 E=7 
C=3 P=4 I=6 S=5 F=9 U=2 N=8 T=1 R=0 E=7 
C=3 P=4 I=6 S=8 F=9 U=2 N=5 T=1 R=0 E=7 
C=3 P=2 I=6 S=7 F=9 U=8 N=5 T=1 R=0 E=4 
C=3 P=2 I=6 S=5 F=9 U=8 N=7 T=1 R=0 E=4 
C=2 P=3 I=7 S=6 F=9 U=8 N=5 T=1 R=0 E=4 
C=2 P=3 I=7 S=4 F=9 U=6 N=8 T=1 R=0 E=5 
C=2 P=3 I=7 S=5 F=9 U=4 N=8 T=1 R=0 E=6 
C=2 P=3 I=7 S=8 F=9 U=4 N=5 T=1 R=0 E=6 
C=2 P=3 I=7 S=8 F=9 U=6 N=4 T=1 R=0 E=5 
C=2 P=3 I=7 S=5 F=9 U=8 N=6 T=1 R=0 E=4 
C=7 P=3 I=2 S=5 F=9 U=8 N=6 T=1 R=0 E=4 
C=7 P=3 I=2 S=6 F=9 U=8 N=5 T=1 R=0 E=4 
C=6 P=2 I=3 S=7 F=9 U=8 N=5 T=1 R=0 E=4 
C=6 P=2 I=3 S=5 F=9 U=8 N=7 T=1 R=0 E=4 
C=7 P=3 I=2 S=4 F=9 U=6 N=8 T=1 R=0 E=5 
C=7 P=3 I=2 S=8 F=9 U=6 N=4 T=1 R=0 E=5 
C=7 P=4 I=2 S=8 F=9 U=6 N=3 T=1 R=0 E=5 
C=7 P=8 I=2 S=4 F=9 U=6 N=3 T=1 R=0 E=5 
C=7 P=8 I=2 S=3 F=9 U=6 N=4 T=1 R=0 E=5 
C=7 P=4 I=2 S=3 F=9 U=6 N=8 T=1 R=0 E=5 
C=7 P=6 I=2 S=3 F=9 U=8 N=5 T=1 R=0 E=4 
C=7 P=5 I=2 S=3 F=9 U=8 N=6 T=1 R=0 E=4 
C=7 P=8 I=2 S=3 F=9 U=4 N=5 T=1 R=0 E=6 
C=7 P=8 I=2 S=5 

In [172]:
"""OR-Tools solution to the N-queens problem."""
import sys
import time
from ortools.sat.python import cp_model


class NQueenSolutionPrinter(cp_model.CpSolverSolutionCallback):
    """Print intermediate solutions."""

    def __init__(self, queens):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self.__queens = queens
        self.__solution_count = 0
        self.__start_time = time.time()

    def solution_count(self):
        return self.__solution_count

    def on_solution_callback(self):
        current_time = time.time()
        print('Solution %i, time = %f s' %
              (self.__solution_count, current_time - self.__start_time))
        self.__solution_count += 1

        all_queens = range(len(self.__queens))
        for i in all_queens:
            for j in all_queens:
                if self.Value(self.__queens[j]) == i:
                    # There is a queen in column j, row i.
                    print('Q', end=' ')
                else:
                    print('_', end=' ')
            print()
        print()



def main(board_size):
    # Creates the solver.
    model = cp_model.CpModel()

    # Creates the variables.
    # The array index is the column, and the value is the row.
    queens = [
        model.NewIntVar(0, board_size - 1, 'x%i' % i) for i in range(board_size)
    ]

    # Creates the constraints.
    # All rows must be different.
    model.AddAllDifferent(queens)

    # All columns must be different because the indices of queens are all
    # different.

    # No two queens can be on the same diagonal.
    model.AddAllDifferent(queens[i] + i for i in range(board_size))
    model.AddAllDifferent(queens[i] - i for i in range(board_size))

    # Solve the model.
    solver = cp_model.CpSolver()
    solution_printer = NQueenSolutionPrinter(queens)
    solver.parameters.enumerate_all_solutions = True
    solver.Solve(model, solution_printer)

    # Statistics.
    print('\nStatistics')
    print(f'  conflicts      : {solver.NumConflicts()}')
    print(f'  branches       : {solver.NumBranches()}')
    print(f'  wall time      : {solver.WallTime()} s')
    print(f'  solutions found: {solution_printer.solution_count()}')


if __name__ == '__main__':
    # By default, solve the 8x8 problem.
    size = 8
#     if len(sys.argv) > 1:
#         size = int(sys.argv[1])
    main(size)

Solution 0, time = 0.004020 s
_ _ _ Q _ _ _ _ 
_ _ _ _ _ _ Q _ 
_ _ _ _ Q _ _ _ 
_ _ Q _ _ _ _ _ 
Q _ _ _ _ _ _ _ 
_ _ _ _ _ Q _ _ 
_ _ _ _ _ _ _ Q 
_ Q _ _ _ _ _ _ 

Solution 1, time = 0.007987 s
_ _ _ _ _ _ _ Q 
_ Q _ _ _ _ _ _ 
_ _ _ _ Q _ _ _ 
_ _ Q _ _ _ _ _ 
Q _ _ _ _ _ _ _ 
_ _ _ _ _ _ Q _ 
_ _ _ Q _ _ _ _ 
_ _ _ _ _ Q _ _ 

Solution 2, time = 0.010986 s
_ _ _ Q _ _ _ _ 
_ _ _ _ _ _ _ Q 
_ _ _ _ Q _ _ _ 
_ _ Q _ _ _ _ _ 
Q _ _ _ _ _ _ _ 
_ _ _ _ _ _ Q _ 
_ Q _ _ _ _ _ _ 
_ _ _ _ _ Q _ _ 

Solution 3, time = 0.017056 s
_ _ _ _ _ _ Q _ 
_ Q _ _ _ _ _ _ 
_ _ _ _ _ Q _ _ 
_ _ Q _ _ _ _ _ 
Q _ _ _ _ _ _ _ 
_ _ _ Q _ _ _ _ 
_ _ _ _ _ _ _ Q 
_ _ _ _ Q _ _ _ 

Solution 4, time = 0.022266 s
_ _ _ Q _ _ _ _ 
_ _ _ _ _ Q _ _ 
_ _ _ _ _ _ _ Q 
_ _ Q _ _ _ _ _ 
Q _ _ _ _ _ _ _ 
_ _ _ _ _ _ Q _ 
_ _ _ _ Q _ _ _ 
_ Q _ _ _ _ _ _ 

Solution 5, time = 0.024760 s
Q _ _ _ _ _ _ _ 
_ _ _ _ _ Q _ _ 
_ _ _ _ _ _ _ Q 
_ _ Q _ _ _ _ _ 
_ _ _ _ _ _ Q _ 
_ _ _ Q _ _ _ _ 
_ Q _ _ _ _ _ _ 
_ _ _ _ Q _ _ _ 

_ _ Q _ _ _ _ _ 
_ _ _ _ _ Q _ _ 
_ Q _ _ _ _ _ _ 
_ _ _ _ _ _ Q _ 
_ _ _ _ Q _ _ _ 

Solution 81, time = 0.375299 s
_ _ _ _ _ Q _ _ 
_ _ Q _ _ _ _ _ 
Q _ _ _ _ _ _ _ 
_ _ _ _ _ _ _ Q 
_ _ _ _ Q _ _ _ 
_ Q _ _ _ _ _ _ 
_ _ _ Q _ _ _ _ 
_ _ _ _ _ _ Q _ 

Solution 82, time = 0.376301 s
_ _ _ _ _ _ Q _ 
_ _ Q _ _ _ _ _ 
Q _ _ _ _ _ _ _ 
_ _ _ _ _ Q _ _ 
_ _ _ _ _ _ _ Q 
_ _ _ _ Q _ _ _ 
_ Q _ _ _ _ _ _ 
_ _ _ Q _ _ _ _ 

Solution 83, time = 0.385299 s
_ _ _ _ _ _ Q _ 
_ Q _ _ _ _ _ _ 
_ _ _ Q _ _ _ _ 
Q _ _ _ _ _ _ _ 
_ _ _ _ _ _ _ Q 
_ _ _ _ Q _ _ _ 
_ _ Q _ _ _ _ _ 
_ _ _ _ _ Q _ _ 

Solution 84, time = 0.386299 s
_ _ _ _ _ _ _ Q 
_ Q _ _ _ _ _ _ 
_ _ _ Q _ _ _ _ 
Q _ _ _ _ _ _ _ 
_ _ _ _ _ _ Q _ 
_ _ _ _ Q _ _ _ 
_ _ Q _ _ _ _ _ 
_ _ _ _ _ Q _ _ 

Solution 85, time = 0.393393 s
_ Q _ _ _ _ _ _ 
_ _ _ _ _ _ Q _ 
_ _ Q _ _ _ _ _ 
_ _ _ _ _ Q _ _ 
_ _ _ _ _ _ _ Q 
_ _ _ _ Q _ _ _ 
Q _ _ _ _ _ _ _ 
_ _ _ Q _ _ _ _ 

Solution 86, time = 0.396392 s
_ _ _ Q _ _ _ _ 
_ _ _ _ _ _ Q _ 
_ _ Q _ _

In [173]:
"""Solves a problem with a time limit."""

from ortools.sat.python import cp_model


def SolveWithTimeLimitSampleSat():
    """Minimal CP-SAT example to showcase calling the solver."""
    # Creates the model.
    model = cp_model.CpModel()
    # Creates the variables.
    num_vals = 3
    x = model.NewIntVar(0, num_vals - 1, 'x')
    y = model.NewIntVar(0, num_vals - 1, 'y')
    z = model.NewIntVar(0, num_vals - 1, 'z')
    # Adds an all-different constraint.
    model.Add(x != y)

    # Creates a solver and solves the model.
    solver = cp_model.CpSolver()

    # Sets a time limit of 10 seconds.
    solver.parameters.max_time_in_seconds = 10.0

    status = solver.Solve(model)

    if status == cp_model.OPTIMAL:
        print('x = %i' % solver.Value(x))
        print('y = %i' % solver.Value(y))
        print('z = %i' % solver.Value(z))


SolveWithTimeLimitSampleSat()

x = 1
y = 0
z = 0


In [174]:
"""Code sample that solves a model and displays a small number of solutions."""

from ortools.sat.python import cp_model


class VarArraySolutionPrinterWithLimit(cp_model.CpSolverSolutionCallback):
    """Print intermediate solutions."""

    def __init__(self, variables, limit):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self.__variables = variables
        self.__solution_count = 0
        self.__solution_limit = limit

    def on_solution_callback(self):
        self.__solution_count += 1
        for v in self.__variables:
            print('%s=%i' % (v, self.Value(v)), end=' ')
        print()
        if self.__solution_count >= self.__solution_limit:
            print('Stop search after %i solutions' % self.__solution_limit)
            self.StopSearch()

    def solution_count(self):
        return self.__solution_count


def StopAfterNSolutionsSampleSat():
    """Showcases calling the solver to search for small number of solutions."""
    # Creates the model.
    model = cp_model.CpModel()
    # Creates the variables.
    num_vals = 3
    x = model.NewIntVar(0, num_vals - 1, 'x')
    y = model.NewIntVar(0, num_vals - 1, 'y')
    z = model.NewIntVar(0, num_vals - 1, 'z')

    # Create a solver and solve.
    solver = cp_model.CpSolver()
    solution_printer = VarArraySolutionPrinterWithLimit([x, y, z], 5)
    # Enumerate all solutions.
    solver.parameters.enumerate_all_solutions = True
    # Solve.
    status = solver.Solve(model, solution_printer)
    print('Status = %s' % solver.StatusName(status))
    print('Number of solutions found: %i' % solution_printer.solution_count())
    assert solution_printer.solution_count() == 5


StopAfterNSolutionsSampleSat()

x=0 y=0 z=0 
x=0 y=1 z=0 
x=0 y=1 z=1 
x=0 y=0 z=1 
x=0 y=2 z=1 
Stop search after 5 solutions
Status = FEASIBLE
Number of solutions found: 5


In [175]:
from ortools.sat.python import cp_model

In [176]:
costs = [
    [90, 76, 75, 70],
    [35, 85, 55, 65],
    [125, 95, 90, 105],
    [45, 110, 95, 115],
    [60, 105, 80, 75],
    [45, 65, 110, 95],
]
num_workers = len(costs)
num_tasks = len(costs[0])

team1 = [0, 2, 4]
team2 = [1, 3, 5]
# Maximum total of tasks for any team
team_max = 2

In [178]:
num_workers

6

In [179]:
model = cp_model.CpModel()

In [180]:
x = {}
for worker in range(num_workers):
    for task in range(num_tasks):
        x[worker, task] = model.NewBoolVar(f'x[{worker},{task}]')

In [195]:
# Each worker is assigned to at most one task.
for worker in range(num_workers):
    model.AddAtMostOne(x[worker, task] for task in range(num_tasks))

# Each task is assigned to exactly one worker.
for task in range(num_tasks):
    model.AddExactlyOne(x[worker, task] for worker in range(num_workers))

# Each team takes at most two tasks.
team1_tasks = []
for worker in team1:
    for task in range(num_tasks):
        team1_tasks.append(x[worker, task])
model.Add(sum(team1_tasks) <= team_max)

team2_tasks = []
for worker in team2:
    for task in range(num_tasks):
        team2_tasks.append(x[worker, task])
model.Add(sum(team2_tasks) <= team_max)

In [196]:
team1

[0, 2, 4]

In [208]:
"""Solve a simple assignment problem."""
from ortools.sat.python import cp_model


def main():
    # Data
    costs = [
        [90, 76, 75, 70],
        [35, 85, 55, 65],
        [125, 95, 90, 105],
        [45, 110, 95, 115],
        [60, 105, 80, 75],
        [45, 65, 110, 95],
    ]
    num_workers = len(costs)
    num_tasks = len(costs[0])

    team1 = [0, 2, 4]
    team2 = [1, 3, 5]
    # Maximum total of tasks for any team
    team_max = 2

    # Model
    model = cp_model.CpModel()

    # Variables
    x = {}
    for worker in range(num_workers):
        for task in range(num_tasks):
            x[worker, task] = model.NewBoolVar(f'x[{worker},{task}]')

    # Constraints
    # Each worker is assigned to at most one task.
    for worker in range(num_workers):
        model.AddAtMostOne(x[worker, task] for task in range(num_tasks))

    # Each task is assigned to exactly one worker.
    for task in range(num_tasks):
        model.AddExactlyOne(x[worker, task] for worker in range(num_workers))

    # Each team takes at most two tasks.
    team1_tasks = []
    for worker in team1:
        for task in range(num_tasks):
            team1_tasks.append(x[worker, task])
    model.Add(sum(team1_tasks) <= team_max)

    team2_tasks = []
    for worker in team2:
        for task in range(num_tasks):
            team2_tasks.append(x[worker, task])
    model.Add(sum(team2_tasks) <= team_max)

    # Objective
    objective_terms = []
    for worker in range(num_workers):
        for task in range(num_tasks):
            objective_terms.append(costs[worker][task] * x[worker, task])
    model.Minimize(sum(objective_terms))

    # Solve
    solver = cp_model.CpSolver()
    status = solver.Solve(model)

    # Print solution.
    if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
        print(f'Total cost = {solver.ObjectiveValue()}\n')
        for worker in range(num_workers):
            for task in range(num_tasks):
                if solver.BooleanValue(x[worker, task]):
                    print(f'Worker {worker} assigned to task {task}.' +
                          f' Cost = {costs[worker][task]}')
    else:
        print('No solution found.')


if __name__ == '__main__':
    main()

Total cost = 250.0

Worker 0 assigned to task 3. Cost = 70
Worker 1 assigned to task 0. Cost = 35
Worker 4 assigned to task 2. Cost = 80
Worker 5 assigned to task 1. Cost = 65


In [209]:
"""MIP example that solves an assignment problem."""
from ortools.linear_solver import pywraplp


def main():
    # Data
    costs = [
        [90, 76, 75, 70],
        [35, 85, 55, 65],
        [125, 95, 90, 105],
        [45, 110, 95, 115],
        [60, 105, 80, 75],
        [45, 65, 110, 95],
    ]
    num_workers = len(costs)
    num_tasks = len(costs[0])

    team1 = [0, 2, 4]
    team2 = [1, 3, 5]
    # Maximum total of tasks for any team
    team_max = 2

    # Solver
    # Create the mip solver with the SCIP backend.
    solver = pywraplp.Solver.CreateSolver('SCIP')
    if not solver:
        return

    # Variables
    # x[i, j] is an array of 0-1 variables, which will be 1
    # if worker i is assigned to task j.
    x = {}
    for worker in range(num_workers):
        for task in range(num_tasks):
            x[worker, task] = solver.BoolVar(f'x[{worker},{task}]')

    # Constraints
    # Each worker is assigned at most 1 task.
    for worker in range(num_workers):
        solver.Add(
            solver.Sum([x[worker, task] for task in range(num_tasks)]) <= 1)

    # Each task is assigned to exactly one worker.
    for task in range(num_tasks):
        solver.Add(
            solver.Sum([x[worker, task] for worker in range(num_workers)]) == 1)

    # Each team takes at most two tasks.
    team1_tasks = []
    for worker in team1:
        for task in range(num_tasks):
            team1_tasks.append(x[worker, task])
    solver.Add(solver.Sum(team1_tasks) <= team_max)

    team2_tasks = []
    for worker in team2:
        for task in range(num_tasks):
            team2_tasks.append(x[worker, task])
    solver.Add(solver.Sum(team2_tasks) <= team_max)

    # Objective
    objective_terms = []
    for worker in range(num_workers):
        for task in range(num_tasks):
            objective_terms.append(costs[worker][task] * x[worker, task])
    solver.Minimize(solver.Sum(objective_terms))

    # Solve
    status = solver.Solve()

    # Print solution.
    if status == pywraplp.Solver.OPTIMAL or status == pywraplp.Solver.FEASIBLE:
        print(f'Total cost = {solver.Objective().Value()}\n')
        for worker in range(num_workers):
            for task in range(num_tasks):
                if x[worker, task].solution_value() > 0.5:
                    print(f'Worker {worker} assigned to task {task}.' +
                          f' Cost = {costs[worker][task]}')
    else:
        print('No solution found.')
    print(f'Time = {solver.WallTime()} ms')


if __name__ == '__main__':
    main()

Total cost = 249.99999999999997

Worker 0 assigned to task 2. Cost = 75
Worker 1 assigned to task 0. Cost = 35
Worker 4 assigned to task 3. Cost = 75
Worker 5 assigned to task 1. Cost = 65
Time = 9 ms


In [222]:
model = cp_model.CpModel()

In [223]:
costs = [
    [90, 76, 75, 70, 50, 74, 12, 68],
    [35, 85, 55, 65, 48, 101, 70, 83],
    [125, 95, 90, 105, 59, 120, 36, 73],
    [45, 110, 95, 115, 104, 83, 37, 71],
    [60, 105, 80, 75, 59, 62, 93, 88],
    [45, 65, 110, 95, 47, 31, 81, 34],
    [38, 51, 107, 41, 69, 99, 115, 48],
    [47, 85, 57, 71, 92, 77, 109, 36],
    [39, 63, 97, 49, 118, 56, 92, 61],
    [47, 101, 71, 60, 88, 109, 52, 90],
]
num_workers = len(costs)
num_tasks = len(costs[0])

task_sizes = [10, 7, 3, 12, 15, 4, 11, 5]
# Maximum total of task sizes for any worker
total_size_max = 15

In [224]:
model = cp_model.CpModel()

In [225]:
x = {}
for worker in range(num_workers):
    for task in range(num_tasks):
        x[worker, task] = model.NewBoolVar(f'x[{worker},{task}]')

In [226]:
# Each worker is assigned to at most one task.
for worker in range(num_workers):
    model.Add(
        sum(task_sizes[task] * x[worker, task]
            for task in range(num_tasks)) <= total_size_max)

# Each task is assigned to exactly one worker.
for task in range(num_tasks):
    model.AddExactlyOne(x[worker, task] for worker in range(num_workers))

In [227]:
objective_terms = []
for worker in range(num_workers):
    for task in range(num_tasks):
        objective_terms.append(costs[worker][task] * x[worker, task])
model.Minimize(sum(objective_terms))

In [228]:
solver = cp_model.CpSolver()
status = solver.Solve(model)

Total cost = 326.0

Worker 0 assigned to task 6. Cost = 12
Worker 1 assigned to task 0. Cost = 35
Worker 1 assigned to task 2. Cost = 55
Worker 4 assigned to task 4. Cost = 59
Worker 5 assigned to task 5. Cost = 31
Worker 5 assigned to task 7. Cost = 34
Worker 6 assigned to task 1. Cost = 51
Worker 8 assigned to task 3. Cost = 49


###  真实case 发红包的场景  demo

In [54]:
import numpy as np
import pandas as pd

In [55]:
a=np.random.rand(20000,3).astype('float16')
data=pd.DataFrame(a,columns=["outcome0","outcome1","outcome2"])

In [56]:
data.shape

(20000, 3)

In [57]:
data.head()

,outcome0,outcome1,outcome2
0,0.985840,0.638184,0.334961
1,0.784668,0.327393,0.449951
2,0.501465,0.824219,0.855469
3,0.583008,0.663574,0.144531
4,0.061340,0.467773,0.212402


In [58]:
red_packet=[1,2,3]

In [59]:
data["uplift0"]=0
data["uplift1"]=(data["outcome1"]-data["outcome0"])/(red_packet[1]-red_packet[0])  #使用斜率代替uplift
data["uplift2"]=(data["outcome2"]-data["outcome1"])/(red_packet[2]-red_packet[1])  #使用斜率代替uplift

In [60]:
data.head()

,outcome0,outcome1,outcome2,uplift0,uplift1,uplift2
0,0.985840,0.638184,0.334961,0,-0.347656,-0.303223
1,0.784668,0.327393,0.449951,0,-0.457275,0.122559
2,0.501465,0.824219,0.855469,0,0.322754,0.031250
3,0.583008,0.663574,0.144531,0,0.080566,-0.519043
4,0.061340,0.467773,0.212402,0,0.406494,-0.255371


In [61]:
outcome=data[["outcome0","outcome1","outcome2"]].values
outcome=outcome.tolist()

In [62]:
costs=data[["uplift0","uplift1","uplift2"]].values
costs=costs.tolist()

In [63]:
# costs

In [64]:
# outcome

In [65]:
num_workers = len(costs)
num_tasks = len(costs[0])

In [66]:
from ortools.sat.python import cp_model
model = cp_model.CpModel()

In [67]:
x = {}
for worker in range(num_workers):
    for task in range(num_tasks):
        x[worker, task] = model.NewBoolVar(f'x[{worker},{task}]')

In [68]:
num_workers,num_tasks

(20000, 3)

In [69]:
# Each worker is assigned to at most one task.
# 每个用户只能发一个红包
for worker in range(num_workers):
    model.AddAtMostOne(x[worker, task] for task in range(num_tasks))


# # Each task is assigned to exactly one worker.
# for task in range(num_tasks):
#     model.AddExactlyOne(x[worker, task] for worker in range(num_workers))

In [70]:
# 添加红包金额的限制
# outcome

In [71]:
red_limit=10000

In [72]:
all_red=[]
for worker in range(num_workers):
    for task in range(num_tasks):
        all_red.append(int(outcome[worker][task]*100)*red_packet[task]*x[worker,task])  # *100是因为该求解器值支持整数
model.Add(sum(all_red)<=(red_limit*100))    #限制条件也乘以100


In [73]:
objective_terms = []
for worker in range(num_workers):
    for task in range(num_tasks):
        objective_terms.append(costs[worker][task] * x[worker, task])
model.Maximize(sum(objective_terms))

In [74]:
solver = cp_model.CpSolver()
status = solver.Solve(model)

In [75]:
result=dict()

In [76]:
if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    print(f'Total cost = {solver.ObjectiveValue()}\n')
    for worker in range(num_workers):
        for task in range(num_tasks):
            if solver.BooleanValue(x[worker, task]):
                result[worker]=red_packet[task]
                print(f'Worker {worker} assigned to task {task}.' +
                      f' Cost = {costs[worker][task]}')
else:
    print('No solution found.')

Total cost = 3542.4487686157227

Worker 4 assigned to task 1. Cost = 0.406494140625
Worker 6 assigned to task 1. Cost = 0.62646484375
Worker 7 assigned to task 1. Cost = 0.41015625
Worker 15 assigned to task 2. Cost = 0.44921875
Worker 19 assigned to task 1. Cost = 0.471923828125
Worker 21 assigned to task 1. Cost = 0.65234375
Worker 22 assigned to task 1. Cost = 0.373779296875
Worker 24 assigned to task 1. Cost = 0.6357421875
Worker 31 assigned to task 1. Cost = 0.280029296875
Worker 37 assigned to task 2. Cost = 0.48388671875
Worker 41 assigned to task 1. Cost = 0.80224609375
Worker 42 assigned to task 2. Cost = 0.88818359375
Worker 43 assigned to task 1. Cost = 0.85546875
Worker 46 assigned to task 1. Cost = 0.4365234375
Worker 47 assigned to task 1. Cost = 0.8583984375
Worker 48 assigned to task 2. Cost = 0.38671875
Worker 49 assigned to task 1. Cost = 0.67578125
Worker 52 assigned to task 1. Cost = 0.40771484375
Worker 56 assigned to task 1. Cost = 0.1424560546875
Worker 58 assign

Worker 3321 assigned to task 2. Cost = 0.1787109375
Worker 3322 assigned to task 1. Cost = 0.6826171875
Worker 3324 assigned to task 1. Cost = 0.57373046875
Worker 3325 assigned to task 1. Cost = 0.90478515625
Worker 3326 assigned to task 1. Cost = 0.83544921875
Worker 3329 assigned to task 2. Cost = 0.625
Worker 3330 assigned to task 1. Cost = 0.373779296875
Worker 3331 assigned to task 2. Cost = 0.71142578125
Worker 3332 assigned to task 1. Cost = 0.7138671875
Worker 3334 assigned to task 1. Cost = 0.8994140625
Worker 3335 assigned to task 1. Cost = 0.52783203125
Worker 3336 assigned to task 1. Cost = 0.37158203125
Worker 3338 assigned to task 1. Cost = 0.349609375
Worker 3347 assigned to task 2. Cost = 0.4453125
Worker 3349 assigned to task 2. Cost = 0.841796875
Worker 3353 assigned to task 1. Cost = 0.66455078125
Worker 3361 assigned to task 2. Cost = 0.69775390625
Worker 3367 assigned to task 2. Cost = 0.69140625
Worker 3368 assigned to task 2. Cost = 0.2880859375
Worker 3374 assi

Worker 4067 assigned to task 1. Cost = 0.44189453125
Worker 4078 assigned to task 1. Cost = 0.798828125
Worker 4082 assigned to task 1. Cost = 0.03179931640625
Worker 4085 assigned to task 1. Cost = 0.5478515625
Worker 4086 assigned to task 1. Cost = 0.56884765625
Worker 4087 assigned to task 1. Cost = 0.5263671875
Worker 4089 assigned to task 1. Cost = 0.548828125
Worker 4093 assigned to task 1. Cost = 0.44921875
Worker 4094 assigned to task 2. Cost = 0.8818359375
Worker 4096 assigned to task 1. Cost = 0.463134765625
Worker 4097 assigned to task 1. Cost = 0.7822265625
Worker 4099 assigned to task 1. Cost = 0.476806640625
Worker 4103 assigned to task 1. Cost = 0.45703125
Worker 4107 assigned to task 1. Cost = 0.446044921875
Worker 4111 assigned to task 1. Cost = 0.5537109375
Worker 4113 assigned to task 1. Cost = 0.92822265625
Worker 4116 assigned to task 1. Cost = 0.67724609375
Worker 4126 assigned to task 1. Cost = 0.7548828125
Worker 4127 assigned to task 1. Cost = 0.11419677734375


Worker 6933 assigned to task 2. Cost = 0.83251953125
Worker 6936 assigned to task 1. Cost = 0.398681640625
Worker 6937 assigned to task 2. Cost = 0.260009765625
Worker 6940 assigned to task 1. Cost = 0.2078857421875
Worker 6942 assigned to task 1. Cost = 0.37841796875
Worker 6945 assigned to task 1. Cost = 0.11474609375
Worker 6950 assigned to task 1. Cost = 0.857421875
Worker 6951 assigned to task 1. Cost = 0.1607666015625
Worker 6952 assigned to task 2. Cost = 0.26708984375
Worker 6953 assigned to task 1. Cost = 0.646484375
Worker 6957 assigned to task 2. Cost = 0.3486328125
Worker 6960 assigned to task 1. Cost = 0.1607666015625
Worker 6962 assigned to task 2. Cost = 0.833984375
Worker 6966 assigned to task 1. Cost = 0.564453125
Worker 6967 assigned to task 1. Cost = 0.2322998046875
Worker 6968 assigned to task 1. Cost = 0.41357421875
Worker 6972 assigned to task 2. Cost = 0.5595703125
Worker 6976 assigned to task 1. Cost = 0.609375
Worker 6978 assigned to task 2. Cost = 0.7934570312

Worker 7709 assigned to task 1. Cost = 0.2445068359375
Worker 7710 assigned to task 1. Cost = 0.2880859375
Worker 7713 assigned to task 2. Cost = 0.8447265625
Worker 7719 assigned to task 1. Cost = 0.52099609375
Worker 7722 assigned to task 1. Cost = 0.419921875
Worker 7723 assigned to task 2. Cost = 0.50390625
Worker 7729 assigned to task 1. Cost = 0.346923828125
Worker 7730 assigned to task 1. Cost = 0.44580078125
Worker 7739 assigned to task 2. Cost = 0.57177734375
Worker 7742 assigned to task 1. Cost = 0.7626953125
Worker 7749 assigned to task 1. Cost = 0.464599609375
Worker 7752 assigned to task 1. Cost = 0.404296875
Worker 7757 assigned to task 2. Cost = 0.353515625
Worker 7762 assigned to task 1. Cost = 0.367431640625
Worker 7767 assigned to task 2. Cost = 0.74462890625
Worker 7772 assigned to task 1. Cost = 0.638671875
Worker 7778 assigned to task 2. Cost = 0.80859375
Worker 7779 assigned to task 2. Cost = 0.55419921875
Worker 7786 assigned to task 1. Cost = 0.24365234375
Worke

Worker 10797 assigned to task 1. Cost = 0.54541015625
Worker 10799 assigned to task 1. Cost = 0.28076171875
Worker 10801 assigned to task 1. Cost = 0.5029296875
Worker 10802 assigned to task 2. Cost = 0.1878662109375
Worker 10806 assigned to task 1. Cost = 0.29443359375
Worker 10807 assigned to task 1. Cost = 0.26708984375
Worker 10814 assigned to task 1. Cost = 0.354736328125
Worker 10816 assigned to task 2. Cost = 0.611328125
Worker 10826 assigned to task 1. Cost = 0.37841796875
Worker 10828 assigned to task 1. Cost = 0.60498046875
Worker 10831 assigned to task 2. Cost = 0.232666015625
Worker 10836 assigned to task 1. Cost = 0.1826171875
Worker 10840 assigned to task 1. Cost = 0.5556640625
Worker 10842 assigned to task 1. Cost = 0.45751953125
Worker 10843 assigned to task 2. Cost = 0.6728515625
Worker 10844 assigned to task 1. Cost = 0.8271484375
Worker 10845 assigned to task 2. Cost = 0.5517578125
Worker 10849 assigned to task 1. Cost = 0.58251953125
Worker 10850 assigned to task 2.

Worker 11564 assigned to task 1. Cost = 0.84033203125
Worker 11567 assigned to task 1. Cost = 0.68994140625
Worker 11569 assigned to task 2. Cost = 0.1029052734375
Worker 11570 assigned to task 1. Cost = 0.37939453125
Worker 11572 assigned to task 1. Cost = 0.4931640625
Worker 11573 assigned to task 2. Cost = 0.60400390625
Worker 11578 assigned to task 2. Cost = 0.4072265625
Worker 11580 assigned to task 1. Cost = 0.283203125
Worker 11584 assigned to task 1. Cost = 0.212646484375
Worker 11586 assigned to task 2. Cost = 0.439453125
Worker 11587 assigned to task 1. Cost = 0.560546875
Worker 11588 assigned to task 1. Cost = 0.3212890625
Worker 11591 assigned to task 1. Cost = 0.66552734375
Worker 11592 assigned to task 1. Cost = 0.90234375
Worker 11593 assigned to task 1. Cost = 0.258544921875
Worker 11597 assigned to task 1. Cost = 0.5712890625
Worker 11599 assigned to task 1. Cost = 0.611328125
Worker 11604 assigned to task 1. Cost = 0.501953125
Worker 11608 assigned to task 2. Cost = 0

Worker 15239 assigned to task 1. Cost = 0.68408203125
Worker 15241 assigned to task 2. Cost = 0.55615234375
Worker 15246 assigned to task 1. Cost = 0.157470703125
Worker 15247 assigned to task 2. Cost = 0.88427734375
Worker 15248 assigned to task 1. Cost = 0.458740234375
Worker 15251 assigned to task 1. Cost = 0.7783203125
Worker 15255 assigned to task 1. Cost = 0.38232421875
Worker 15261 assigned to task 1. Cost = 0.25927734375
Worker 15262 assigned to task 1. Cost = 0.388916015625
Worker 15267 assigned to task 2. Cost = 0.55419921875
Worker 15270 assigned to task 2. Cost = 0.6962890625
Worker 15274 assigned to task 1. Cost = 0.77978515625
Worker 15279 assigned to task 1. Cost = 0.473388671875
Worker 15280 assigned to task 2. Cost = 0.83837890625
Worker 15281 assigned to task 1. Cost = 0.658203125
Worker 15283 assigned to task 1. Cost = 0.55810546875
Worker 15295 assigned to task 2. Cost = 0.87060546875
Worker 15296 assigned to task 1. Cost = 0.5927734375
Worker 15298 assigned to task